# Westeros Tutorial: Adding learning parameters

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

from message_ix.tools.add_learning import add_learning


%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "Platform.java", line 110, in at.ac.iiasa.ixmp.Platform.<init>
oracle.net.ns.oracle.net.ns.NetException: oracle.net.ns.NetException: Unknown host specified 

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "Platform.java", line 110, in at.ac.iiasa.ixmp.Platform.<init>
java.sql.java.sql.SQLRecoverableException: java.sql.SQLRecoverableException: IO Error: Unknown host specified 

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "Platform.java", line 110, in at.ac.iiasa.ixmp.Platform.<init>
Exception: Java Exception

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\pratama\Anaconda3\envs\message_env\lib\site-packages\ixmp\backend\jdbc.py", line 269, in __init__
    self.jobj = java.Platform("Python", properties)
com.zaxxer.hikari.pool.HikariPool.com.z

## Making a clone of the existing scenario 'baseline'

In [ ]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="emission_bound")
scen = base.clone(
    model,
    "add_learning",
    "introducing learning parameters",
    keep_solution=False,
)
scen.check_out()

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

## Add learning parameters to the scenario

To add learning parameters to the scenario, one needs to include `add_learning(Scenario)` to the script, with `Scenario` is the name of scenario object that you want to run. In this example, we want to run `scen`, thus:

In [ ]:
add_learning(scen)

## Time to Solve the Model

In [ ]:
scen.commit(comment="Introducing add_learning feature to add technology learning parameters")

In [ ]:
scen.solve(gams_args =["--learningmode=1"])

In [ ]:
scen.var("OBJ")["lvl"]

## Plotting Results

In [ ]:
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

rep = Reporter.from_scenario(scen)
prepare_plots(rep)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
rep.set_filters(t=["coal_ppl", "wind_ppl"])
rep.get("plot activity")

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
rep.get("plot capacity")

### Electricity Price

And how much does the electricity cost? These prices are in fact **shadow prices** taken from the **dual variables** of the model solution. They reflect the marginal cost of electricity generation (i.e., the additional cost of the system for supplying one more unit of electricity), which is in fact the marginal cost of the most expensive generator.  

Notice the drop in the price in the period of 710? 

The share of the activity of the coal power plant is in the period of 710 higher then the share in the period of 700. With lower electricity costs of the coal power plant, this leads to reduced electricity costs. To stay within the emission bounds with an increased electricity demand, in the upcoming period of 720, the capacity of the wind turbines increases drastically, while the capacity of the coal power plants remains the same. This results in the increased electricity price.

In [ ]:
rep.set_filters(t=None, c=["light"])
rep.get("plot prices")

## Close the connection to the database

In [ ]:
mp.close_db()